We would like to find a mathematical formula that will create a new feature from the ones we have - and therefore give the machine learning algorithms more to work with. In the case of the Santander competition, this is dificult because the features are already well pre-processed. In other words, they are already statistically independent and seem to contain little, if any, redundant information (as if principal component analysis has already been performed). Instead, we can turn to generically engineered formulas to create new features.

Specificaly, if we use 199 of the 200 features to create an estimation of the missing feature - this will be a imperfect estimation, because as noted above, the features do not contain redundant information. Nevertheless, this poor estimation can be considered to be a "new feature" or a new classification of the current 199 features as being a member of the missing feature target. This can be repeated for every feature, giving us 200 new features to add to the training (and testing) sets.

This kernel demonstrates this, and uses "gplearn" which extends the scikit-learn machine learning library to perform Genetic Programming (GP) with symbolic regression.

From the website https://gplearn.readthedocs.io/en/stable/intro.html ...

"Symbolic regression is a machine learning technique that aims to identify an underlying mathematical expression that best describes a relationship. It begins by building a population of naive random formulas to represent a relationship between known independent variables and their dependent variable targets in order to predict new data. Each successive generation of programs is then evolved from the one that came before it by selecting the fittest individuals from the population to undergo genetic operations."

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
sns.set_style('whitegrid')

import time

import lightgbm as lgb

from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.model_selection import GridSearchCV

# GENETIC ALGORITHM
import gplearn
from gplearn.genetic import SymbolicTransformer, SymbolicRegressor
from gplearn.functions import make_function
from gplearn.fitness import make_fitness

In [ ]:
train_file = '../data/train.csv'
test_file = '../data/test.csv'

df_train = pd.read_csv(train_file)
df_test = pd.read_csv(test_file)

np.random.seed(42)

In [ ]:

X1 = df_train.drop(['ID_code','target'],axis=1)
X_test1 = df_test.drop('ID_code',axis=1)

# Create a fitness function that is the mean absolute percentage error
def _my_fit(y, y_pred, w):
    diffs = np.abs(y - y_pred)  
    return 100. * np.average(diffs, weights=w)



my_fit = make_fitness(_my_fit, greater_is_better=False)

In [ ]:
# Choose the mathematical functions we will combine together
function_set = ['add', 'sub', 'mul', 'div', 'log', 
                'sqrt', 'log', 'abs', 'neg', 'inv', 
                'max', 'min', 'sin', 'cos', 'tan' ] 

# Create the genetic learning regressor
gp = SymbolicRegressor(function_set=function_set, metric = my_fit,
                       verbose=1, generations = 3, random_state=0, n_jobs=-1)

In [ ]:
# Using NUMPY structures, remove one feature (column of data) at a time from the training set
# Use that removed column as the target for the algorithm
# Use the genetically engineered formula to create the new feature
# Do this for both the training set and the test set

X1a = np.array(X1)
sam = X1a.shape[0]
col = X1a.shape[1]
X2a = np.zeros((sam, col))

X_test1a = np.array(X_test1)
sam_test = X_test1a.shape[0]
col_test = X_test1a.shape[1]
X_test2a = np.zeros((sam_test, col_test))

for i in range(col) :
    X = np.delete(X1a,i,1)
    y = X1a[:,i]
    gp.fit(X, y) 
    X2a[:,i] = gp.predict(X)
    X = np.delete(X_test1a, i, 1)
    X_test2a[:,i] = gp.predict(X)
    
X2 = pd.DataFrame(X2a)
X_test2 = pd.DataFrame(X_test2a) 

In [ ]:
# Add the new features to the existing 200 features
X = pd.concat([X1, X2], axis=1, sort=False) 
X_test = pd.concat([X_test1, X_test2], axis=1, sort=False)  
y = train_df['target']

In [ ]:
params = {
    'num_leaves': 13,
    'min_data_in_leaf': 80,
    'objective': 'binary',
    'max_depth': -1,
    'learning_rate': 0.01,
    'boosting': 'gbdt',
    'bagging_freq': 5,
    'bagging_fraction': 0.33,
    'feature_fraction': 0.05,
    'metric':'auc',
    'verbosity': 1, 
    'min_sum_hessian_in_leaf': 10.0,
    'num_threads': 12,
    'tree_learner': 'serial',
    'boost_from_average':'false'
}

In [ ]:
n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True, random_state=42)

prediction = np.zeros(len(X_test))
for fold_n, (train_index, valid_index) in enumerate(folds.split(X,y)):
    print('Fold', fold_n, 'started at', time.ctime())
    X_train, X_valid = X.iloc[train_index], X.iloc[valid_index]
    y_train, y_valid = y.iloc[train_index], y.iloc[valid_index]
    
    train_data = lgb.Dataset(X_train, label=y_train)
    valid_data = lgb.Dataset(X_valid, label=y_valid)
        
    model = lgb.train(params,train_data,num_boost_round=20000,
                    valid_sets = [train_data, valid_data],verbose_eval=300,early_stopping_rounds = 200)
             
    y_pred_valid += model.predict(X_valid)/n_fold
    prediction += model.predict(X_test, num_iteration=model.best_iteration)/n_fold 
    
val_score = roc_auc_score(y_valid, y_pred_valid)
print('Validation AUC: ', val_score)

In [ ]:
sub = pd.DataFrame({"ID_code": test_df.ID_code.values})
sub["target"] = prediction
sub.to_csv("../results/gplearn_xgb.csv", index=False)